### **Setup**

In [ ]:
! pip install --upgrade --quiet llama-index llama-index-llms-gemini llama-index-embeddings-gemini llama-index-vector-stores-pinecone pinecone-client

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 215.5/215.5 kB 4.5 MB/s eta 0:00:00


In [ ]:
! wget https://huggingface.co/spaces/rasyosef/RAG-with-Phi-2-and-LangChain/raw/main/Oppenheimer-movie-wiki.txt -P ./data

--2024-05-19 19:10:57--  https://huggingface.co/spaces/rasyosef/RAG-with-Phi-2-and-LangChain/raw/main/Oppenheimer-movie-wiki.txt
Resolving huggingface.co (huggingface.co)... 3.163.189.74, 3.163.189.37, 3.163.189.90, ...
Connecting to huggingface.co (huggingface.co)|3.163.189.74|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 51987 (51K) [text/plain]
Saving to: ‘./data/Oppenheimer-movie-wiki.txt’

Oppenheimer-movie-w 100%[===================>]  50.77K  --.-KB/s    in 0.007s  

2024-05-19 19:10:57 (7.44 MB/s) - ‘./data/Oppenheimer-movie-wiki.txt’ saved [51987/51987]



In [ ]:
import os
from google.colab import userdata

os.environ["GOOGLE_API_KEY"] = userdata.get("GEMINI_API_KEY")
os.environ["PINECONE_API_KEY"] = userdata.get("PINECONE_API_KEY")

### **Load Data**

In [ ]:
from llama_index.core import SimpleDirectoryReader

reader = SimpleDirectoryReader(input_dir="./data")
documents = reader.load_data()

print("Number of Documents:", len(documents))

Number of Documents: 1


In [ ]:
from llama_index.core.node_parser import SentenceSplitter

node_parser = SentenceSplitter(chunk_size=512, chunk_overlap=32)
nodes = node_parser.get_nodes_from_documents(documents)

print("Number of Nodes:", len(nodes))

Number of Nodes: 25


### **Embeddings**

In [ ]:
from llama_index.embeddings.gemini import GeminiEmbedding

embed_model = GeminiEmbedding(
    model="models/embedding-001",
    title="Oppenheimer movie wikipedia",
    embed_batch_size=16
)

### **Pinecone Vector Store**

In [ ]:
from pinecone import Pinecone, ServerlessSpec

pinecone = Pinecone()
pinecone

In [ ]:
INDEX_NAME = "rag"

if INDEX_NAME in pinecone.list_indexes().names():
  pinecone.delete_index(INDEX_NAME)

pinecone.create_index(
    name=INDEX_NAME,
    dimension=768,
    metric="dotproduct",
    spec=ServerlessSpec(cloud="aws", region="us-east-1"),
)

In [ ]:
pinecone_index = pinecone.Index(INDEX_NAME)

In [ ]:
from llama_index.vector_stores.pinecone import PineconeVectorStore
from llama_index.core.indices import VectorStoreIndex
from llama_index.core import StorageContext

vector_store = PineconeVectorStore(pinecone_index=pinecone_index)
storage_context = StorageContext.from_defaults(vector_store=vector_store)

In [ ]:
index = VectorStoreIndex(
    nodes=nodes,
    storage_context=storage_context,
    embed_model=embed_model
)

Upserted vectors:   0%|          | 0/25 [00:00<?, ?it/s]

### **Query Engine**

In [ ]:
from google.generativeai.types import HarmCategory, HarmBlockThreshold

#BLOCK_ONLY_HIGH
safety_settings={
  HarmCategory.HARM_CATEGORY_HATE_SPEECH: HarmBlockThreshold.BLOCK_NONE,
  HarmCategory.HARM_CATEGORY_HARASSMENT: HarmBlockThreshold.BLOCK_NONE,
  HarmCategory.HARM_CATEGORY_SEXUALLY_EXPLICIT: HarmBlockThreshold.BLOCK_NONE,
  HarmCategory.HARM_CATEGORY_DANGEROUS_CONTENT: HarmBlockThreshold.BLOCK_NONE,
}

In [ ]:
from llama_index.llms.gemini import Gemini

llm = Gemini(
    model_name="models/gemini-pro",
    temperature=0,
    max_tokens=256,
    safety_settings=safety_settings
)

In [ ]:
query_engine = index.as_query_engine(
    llm=llm,
    similarity_top_k=3,
)

In [ ]:
response = query_engine.query("What's the name of the actor that played Lewis Strauss?")
response.response

'Robert Downey Jr.'

In [ ]:
response = query_engine.query("Who is Boris Pash?")
response.response

'Boris Pash is a U.S. Army military intelligence officer and commander of the Alsos Mission.'

In [ ]:
response = query_engine.query("How much money did the Oppenheimer movie make at the US and global box office?")
response.response

'As of January 15, 2024, Oppenheimer has grossed $326.4 million in the United States and Canada and $628.9 million in other territories, for a worldwide total of $955.3 million.'

In [ ]:
response = query_engine.query("What score did the Oppenheimer movie get on Rotten Tomatoes and Metacritic?")
response.response

'On Rotten Tomatoes, Oppenheimer received a score of 8.6/10 based on 495 reviews, with 93% of the reviews being positive. On Metacritic, the film received a score of 89 out of 100, based on 69 reviews, indicating "universal acclaim".'

In [ ]:
response = query_engine.query("In the plot of the Oppenheimer movie, why did Lewis Strauss have a grudge against J. Robert Oppenheimer?")
response.response

'Strauss resented Oppenheimer for publicly humiliating him by dismissing his concerns about exporting radioisotopes and for recommending negotiations with the Soviet Union after they successfully detonated their own bomb. He also believed that Oppenheimer denigrated him during a conversation Oppenheimer had with Einstein in 1947.'

In [ ]:
response = query_engine.query("What happened while Oppenheimer was a student at the University of Cambridge?")
response.response

'While studying at the University of Cambridge, Oppenheimer grappled with anxiety and homesickness. He left a poisoned apple for his supervisor, Patrick Blackett, but later retrieved it. Visiting scientist Niels Bohr recommended that Oppenheimer study theoretical physics at the University of Göttingen instead.'

In [ ]:
response = query_engine.query("Which character did Matthias Schweighöfer play in the movie?")
response.response

'The provided context does not mention which character Matthias Schweighöfer played in the movie.'

In [ ]:
response = query_engine.query("Which character did Casey Affleck portray in the movie?")
response.response

'The provided context does not mention Casey Affleck or the character he portrayed in the movie.'

In [ ]:
sources = response.source_nodes
for node in sources:
  print(node.text)
  print("\n\n------------------------------------------\n\n")

len(response.source_nodes)

[54] He wanted to execute a quick tonal shift after the atomic bombings of Hiroshima and Nagasaki, desiring to go from the "highest triumphalism, the highest high, to the lowest low in the shortest amount of screen time possible".[69] For the ending, Nolan chose to make it intentionally vague to be open to interpretation and refrained from being didactic or conveying specific messages in his work. However, he did have the intention to present a "strong set of troubling reverberations at the end".[73]

Casting
Oppenheimer marks the sixth collaboration between Nolan and Murphy, and the first starring Murphy as the lead. To prepare for the role, the actor did what he summarized as "an awful lot of reading" on Oppenheimer's life and had also been inspired by David Bowie's appearance in the 1970s.[75][9][66] Nolan called Murphy one day to ask him to play the part, and Murphy enthusiastically accepted and was excited to play a lead role in a Nolan film. Afterward, Nolan flew to Dublin to mee

5

### **Reranking with ColBERT**

In [ ]:
! pip install --quiet llama-index-postprocessor-colbert-rerank

In [ ]:
from llama_index.postprocessor.colbert_rerank import ColbertRerank

colbert_reranker = ColbertRerank(
    top_n=3,
    model="colbert-ir/colbertv2.0",
    tokenizer="colbert-ir/colbertv2.0",
    keep_retrieval_score=True
)

In [ ]:
reranked_query_engine = index.as_query_engine(
    llm=llm,
    similarity_top_k=16,
    node_postprocessors=[colbert_reranker],
)

In [ ]:
response = reranked_query_engine.query("Which character did Matthias Schweighöfer play in the movie?")
response.response

'Werner Heisenberg'

In [ ]:
response = reranked_query_engine.query("Which character did Casey Affleck portray in the movie?")
response.response

'Boris Pash'

In [ ]:
sources = response.source_nodes
for node in sources:
  print(node.text)
  print("\n\n------------------------------------------\n\n")

len(response.source_nodes)

Oppenheimer is a 2023 epic biographical thriller film written and directed by Christopher Nolan. It stars Cillian Murphy as J. Robert Oppenheimer, the American theoretical physicist credited with being the "father of the atomic bomb" for his role in the Manhattan Project—the World War II undertaking that developed the first nuclear weapons. Based on the 2005 biography American Prometheus by Kai Bird and Martin J. Sherwin, the film chronicles the career of Oppenheimer, with the story predominantly focusing on his studies, his direction of the Manhattan Project during World War II, and his eventual fall from grace due to his 1954 security hearing. The film also stars Emily Blunt as Oppenheimer's wife "Kitty", Matt Damon as head of the Manhattan Project Leslie Groves, Robert Downey Jr. as United States Atomic Energy Commission member Lewis Strauss, and Florence Pugh as Oppenheimer's communist lover Jean Tatlock. The ensemble supporting cast includes Josh Hartnett, Casey Affleck, Rami Male

3